# 1. 바이인코더 예제

바이인코더(Bi-encoder)는 자연어 처리에서 사용되는 신경망 아키텍처 중 하나입니다. 주로 문장 유사도 계산, 의미 검색, 개체명 연결 등의 태스크에 활용됩니다.

바이인코더의 주요 특징은 두 개의 인코더를 사용한다는 점입니다. 일반적으로 두 인코더는 동일한 구조를 가지지만, 서로 다른 가중치를 학습합니다. 각 인코더는 입력 문장을 고정 크기의 벡터로 변환하는 역할을 합니다.

바이인코더의 동작 과정은 다음과 같습니다:

1. 두 개의 입력 문장이 각각의 인코더에 전달됩니다.
2. 각 인코더는 해당 문장을 벡터로 변환합니다. 이 벡터를 임베딩 벡터라고 합니다.
3. 두 임베딩 벡터 사이의 유사도를 계산합니다. 유사도 계산에는 내적(dot product), 코사인 유사도 등의 방법이 사용될 수 있습니다.
4. 계산된 유사도를 기반으로 문장 쌍의 관련성을 판단하거나, 검색 시스템에서 질의와 가장 유사한 문장을 찾는 등의 작업을 수행합니다.

바이인코더의 장점은 다음과 같습니다:

1. 입력 문장을 독립적으로 처리하므로, 실시간 추론 속도가 빠릅니다.
2. 임베딩 벡터를 미리 계산하여 저장해 둘 수 있어, 대규모 코퍼스에 대한 유사도 계산이 효율적입니다.
3. 임베딩 벡터는 고정 크기이므로, 가변 길이 입력을 다룰 수 있습니다.

바이인코더는 트랜스포머 기반 모델(예: BERT)과 함께 사용되어 많은 자연어 처리 태스크에서 좋은 성능을 보여주고 있습니다. 다만, 크로스 인코더(Cross-encoder)와 비교하면 문장 간 상호작용을 직접 모델링하지 않으므로, 정확도 면에서는 다소 낮을 수 있습니다.

In [1]:
from sentence_transformers import SentenceTransformer, util

In [2]:
# 바이인코더 모델 로드
bi_encoder = SentenceTransformer('paraphrase-distilroberta-base-v1')

# 문장 쌍 정의
sentences1 = ['The cat sits outside', 'A man is playing guitar', 'The new movie is awesome']
sentences2 = ['The dog plays in the garden', 'A woman watches TV', 'The new movie is so great']

# 각 문장을 인코딩
embeddings1 = bi_encoder.encode(sentences1, convert_to_tensor=True)
embeddings2 = bi_encoder.encode(sentences2, convert_to_tensor=True)

# 코사인 유사도 계산
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

# 결과 출력
for i in range(len(sentences1)):
    print(f"Sentence 1: {sentences1[i]}")
    print(f"Sentence 2: {sentences2[i]}")
    print(f"Similarity score: {cosine_scores[i][i].item():.4f}")
    print()

Sentence 1: The cat sits outside
Sentence 2: The dog plays in the garden
Similarity score: 0.4579

Sentence 1: A man is playing guitar
Sentence 2: A woman watches TV
Similarity score: 0.1759

Sentence 1: The new movie is awesome
Sentence 2: The new movie is so great
Similarity score: 0.9283



# 2. 크로스인코더 예제
크로스 인코더(Cross-encoder)는 바이인코더와 함께 자연어 처리에서 널리 사용되는 신경망 아키텍처입니다. 크로스 인코더는 두 개의 입력 문장을 함께 처리하여 문장 쌍의 관계를 모델링합니다.

크로스 인코더의 동작 과정은 다음과 같습니다:

1. 두 개의 입력 문장을 하나의 시퀀스로 연결합니다. 이때 문장 사이에 특수 토큰(예: [SEP])을 삽입하여 문장 구분을 나타냅니다.
2. 연결된 시퀀스를 인코더(예: BERT)에 전달하여 문장 쌍에 대한 통합된 표현을 얻습니다.
3. 인코더의 출력을 기반으로 문장 쌍의 관계를 예측하는 분류기(classifier)를 통과시킵니다.
4. 분류기의 출력을 사용하여 문장 쌍의 유사도, 관련성 또는 다른 관계를 판단합니다.

크로스 인코더의 장점은 다음과 같습니다:

1. 두 문장을 함께 처리하므로, 문장 간의 상호작용을 직접 모델링할 수 있습니다.
2. 문장 쌍의 관계를 보다 정확하게 예측할 수 있습니다.
3. 사전 훈련된 언어 모델(예: BERT)의 강력한 성능을 활용할 수 있습니다.

하지만 크로스 인코더는 다음과 같은 단점도 있습니다:

1. 추론 시간이 바이인코더에 비해 느립니다. 각 문장 쌍마다 인코더를 통과해야 하므로, 계산 비용이 높습니다.
2. 대규모 코퍼스에 대한 유사도 계산이 비효율적입니다. 모든 문장 쌍에 대해 인코더를 실행해야 하므로, 시간과 자원이 많이 소모됩니다.

크로스 인코더는 주로 문장 쌍의 관계를 보다 정확하게 예측해야 하는 태스크(예: 자연어 추론, 의미 유사도 계산)에 사용됩니다. 반면, 대규모 코퍼스에 대한 유사도 계산이나 검색 태스크에는 바이인코더가 더 적합합니다.

실제 응용에서는 태스크의 특성과 요구 사항에 따라 바이인코더와 크로스 인코더를 적절히 선택하거나 조합하여 사용합니다.

In [3]:
from sentence_transformers import CrossEncoder

In [4]:
# 크로스인코더 모델 로드
cross_encoder = CrossEncoder('cross-encoder/stsb-roberta-base')

# 문장 쌍 정의
sentence_pairs = [
    ('The cat sits outside', 'The dog plays in the garden'),
    ('A man is playing guitar', 'A woman watches TV'),
    ('The new movie is awesome', 'The new movie is so great')
]

# 각 문장 쌍에 대한 유사도 계산
similarity_scores = cross_encoder.predict(sentence_pairs)

# 결과 출력
for i, pair in enumerate(sentence_pairs):
    print(f"Sentence 1: {pair[0]}")
    print(f"Sentence 2: {pair[1]}")
    print(f"Similarity score: {similarity_scores[i]:.4f}")
    print()

Sentence 1: The cat sits outside
Sentence 2: The dog plays in the garden
Similarity score: 0.0495

Sentence 1: A man is playing guitar
Sentence 2: A woman watches TV
Similarity score: 0.0022

Sentence 1: The new movie is awesome
Sentence 2: The new movie is so great
Similarity score: 0.9920



# 3. 바이인코더 vs 크로스인코더
바이인코더와 크로스 인코더는 자연어 처리에서 문장 쌍의 관계를 모델링하는 데 사용되는 두 가지 주요 아키텍처입니다. 다음은 두 아키텍처의 비교입니다:

1. 구조 및 동작 방식:
- 바이인코더: 두 개의 독립적인 인코더를 사용하여 각 문장을 별도로 처리합니다. 인코딩된 벡터 사이의 유사도를 계산하여 문장 쌍의 관계를 판단합니다.
- 크로스 인코더: 두 문장을 하나의 시퀀스로 연결하여 단일 인코더에 전달합니다. 인코더의 출력을 기반으로 문장 쌍의 관계를 예측하는 분류기를 사용합니다.

2. 추론 속도:
- 바이인코더: 각 문장을 독립적으로 인코딩하므로 추론 속도가 빠릅니다. 대규모 코퍼스에 대한 유사도 계산이 효율적입니다.
- 크로스 인코더: 각 문장 쌍마다 인코더를 통과해야 하므로 추론 속도가 상대적으로 느립니다. 대규모 코퍼스에 대한 유사도 계산은 비효율적일 수 있습니다.

3. 정확도:
- 바이인코더: 문장 간 상호작용을 직접 모델링하지 않으므로, 정확도가 크로스 인코더에 비해 다소 낮을 수 있습니다.
- 크로스 인코더: 문장 간 상호작용을 직접 모델링하므로, 문장 쌍의 관계를 보다 정확하게 예측할 수 있습니다.

4. 메모리 사용량:
- 바이인코더: 각 문장에 대한 임베딩 벡터를 저장해야 하므로, 메모리 사용량이 크로스 인코더에 비해 높을 수 있습니다.
- 크로스 인코더: 각 문장 쌍을 개별적으로 처리하므로, 메모리 사용량이 바이인코더에 비해 낮습니다.

5. 적용 분야:
- 바이인코더: 대규모 코퍼스에 대한 유사도 계산, 의미 검색, 개체명 연결 등의 태스크에 적합합니다.
- 크로스 인코더: 문장 쌍의 관계를 보다 정확하게 예측해야 하는 태스크(예: 자연어 추론, 의미 유사도 계산)에 적합합니다.

두 아키텍처는 각각의 장단점이 있으므로, 태스크의 특성과 요구 사항에 따라 적절한 아키텍처를 선택해야 합니다. 경우에 따라서는 두 아키텍처를 조합하여 사용하기도 합니다. 예를 들어, 바이인코더를 사용하여 후보 문장을 필터링한 후, 크로스 인코더를 사용하여 최종 순위를 결정하는 방식 등이 있습니다.